# Author: Seunghee Kim
- Created on: 2024-12-05

In [ ]:
# Notebook Explain
# 기존에 train / valid / test 데이터셋을 구축해서 deberta-v3-xsmall 모델로 train data 학습 후 test dataset에 대해 예측했는데, 정확도 0.99가 나오게 됨.
# 데이터셋의 난이도가 너무 쉽다는 문제점 발생
# 원인 분석
# 원인: train, valid, test 를 이루고 있는 AI-Generated Text가 모두 GPT-4o-mini 모델로 구성됨.
# 즉, GPT-4o-mini 모델로 만들어진 AI-Generated Text로 학습을 했으니 그 패턴을 파악하게 돼서 GPT-4-mini 모델로 만들어진 Test dataset이 난이도가 쉬워서 정확하게 분류가 가능하게 된 것
# 해결 방안: train & valid data와 test data의 distribution을 다르게 하기 위해서, test dataset은 GPT-4o-mini Model뿐만 아니라 최대한 다양한 모델을 이용해서 구축하게 되면 Custom dataset의 난이도가 더 향상될 것이다.

In [ ]:
# train dataset_v2: Human Text 4500개, AI-Text: GPT-4o-mini Text 4500개 총 9000개
# valid dataset_v2: Human Text 450개, AI-Text: GPT-4o-mini Text 450개 총 900개

# test dataset_v2: Human Text 450개, 
# AI-Text:
# 1.'gpt-4o' 50개
# 2.'meta-llama/Llama-3.2-8B-Instruct' 50개
# 3.'Qwen/Qwen2.5-7B-Instruct' 50개
# 4.'Qwen/Qwen2.5-14B-Instruct' 50개
# 5.'Qwen/Qwen2.5-1.5B-Instruct' 50개
# 6.'Qwen/Qwen2.5-0.5B-Instruct' 50개
# 7.'gpt-3.5-turbo' 50개 
# 8.'o1-mini' 50개 
# 9.'gpt-4o-mini' 50개 
# 총 900개

# Import Library

In [2]:
import os
import pandas as pd
from tqdm import tqdm
import yaml
import torch
from transformers import pipeline


/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

# Config & Path

In [ ]:
class CFG:
    SEED = 1
    
    MODEL_1 = "gpt-4o" 
    MODEL_2 = 'meta-llama/Llama-3.1-8B-Instruct'
    MODEL_3 = 'Qwen/Qwen2.5-7B-Instruct'
    MODEL_4 = 'Qwen/Qwen2.5-14B-Instruct'
    MODEL_5 = 'Qwen/Qwen2.5-1.5B-Instruct'
    MODEL_6 = 'Qwen/Qwen2.5-0.5B-Instruct'
    MODEL_7 = 'gpt-3.5-turbo' 
    MODEL_8 = 'o1-mini' 
    MODEL_9 = 'gpt-4o-mini' 
    
    # api key가 담긴 yaml파일 (해당 yaml 파일은 .gitignore에 반드시 추가해야 함!!!)
    API_CONFIG_PATH = './config.yaml'
    
    # 1_dataset_preprocess.ipynb의 결과로 나온 전처리된 Human Dataset 경로
    DF_HUMAN_TEST_PATH = './df_human_test.csv'
    
    # AI-Generated Text까지 포함된 csv파일의 output 경로
    DF_HUMAN_AI_TEST_PATH_1 = './df_human_ai_test_1_gpt-4o.csv'
    DF_HUMAN_AI_TEST_PATH_2 = './df_human_ai_test_2_Llama-3_1-8B-Instruct.csv' 
    DF_HUMAN_AI_TEST_PATH_3 = './df_human_ai_test_3_Qwen2_5-7B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_4 = './df_human_ai_test_4_Qwen2_5-14B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_5 = './df_human_ai_test_5_Qwen2_5-1_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_6 = './df_human_ai_test_6_Qwen2_5-0_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_7 = './df_human_ai_test_7_gpt-3_5_turbo.csv'
    DF_HUMAN_AI_TEST_PATH_8 = './df_human_ai_test_8_o1-mini.csv' 
    DF_HUMAN_AI_TEST_PATH_9 = './df_human_ai_test_9_gpt-4o-mini.csv'
    
    # Final Dataset output 경로 (Human Text, AI Text, Label 존재)
    DF_FINAL_TEST_PATH = './df_final_test_v2.csv'
    
    # CACHE_DIR = '/data/ksh1234/ksh1234/huggingface_cache' 



In [ ]:
# SPLIT TEST DATASET INTO 50 * 9
test_dataset = pd.read_csv(CFG.DF_HUMAN_TEST_PATH)
split_datasets = []
chunk_size = 50

for i in range(0, len(test_dataset), chunk_size):
    split_datasets.append(test_dataset.iloc[i:i + chunk_size])

test_dataset_1 = split_datasets[0]
test_dataset_2 = split_datasets[1]
test_dataset_3 = split_datasets[2]
test_dataset_4 = split_datasets[3]
test_dataset_5 = split_datasets[4]
test_dataset_6 = split_datasets[5]
test_dataset_7 = split_datasets[6]
test_dataset_8 = split_datasets[7]
test_dataset_9 = split_datasets[8]

# Model 2: meta-llama/Llama-3.1-8B-Instruct

In [ ]:
model_id = CFG.MODEL_2
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

test_dataset_2['AI_Generated_Text'] = ""
test_dataset_2['Model'] = ""

for idx, row in tqdm(test_dataset_2.iterrows(), total=len(test_dataset_2)):
    input_text = row['essay_prompt']
    student_grade = row['student_grade']
    school, grade = student_grade.split('_')
    if school == '중등':
        school = school[0]  # 중등인 경우 '중'만 가져옴
    
    # 학교 수준에 따라 다르게 설정하는 시스템 프롬프트
    grade_system_prompt = f"대한민국의 {school}학교 {grade} 학생 수준으로 답하시오. 마크다운 용법을 사용하지 말고 학생이 글을 쓰듯이 답하시오."
    messages = [
        {"role": "system", "content": grade_system_prompt},
        {"role": "user", "content": input_text},
    ]
    
    outputs = pipe(
        messages,
        max_new_tokens=1024,
    )
    
    llm_output = outputs[0]["generated_text"]  
    
    test_dataset_2.at[idx, 'AI_Generated_Text'] = llm_output
    test_dataset_2.at[idx, 'Model'] = model_id

test_dataset_2.to_csv(CFG.DF_HUMAN_AI_TEST_PATH_2, index=False)

In [10]:
import pandas as pd
df = pd.read_csv(CFG.DF_HUMAN_AI_TEST_PATH_2)

def clean_ai_generated_text(text):
    text = text.replace("[{'role': 'system', 'content': '대한민국의 중학교 1학년 학생 수준으로 답하시오. 마크다운 용법을 사용하지 말고 학생이 글을 쓰듯이 답하시오.'}, {'role': 'user', 'content': '", '')
    text = text.rstrip("'}]")
    return text

df['AI_Generated_Text'] = df['AI_Generated_Text'].apply(clean_ai_generated_text)

# 결과 확인
df.to_csv(CFG.DF_HUMAN_AI_TEST_PATH_2, index=False)
df['AI_Generated_Text'][0]

'미래의 도시는 어떤 모습일까요? 기술의 발전으로 인해 자동차가 하늘을 날아다니고 사람들은 로봇과 자연스럽게 대화하며 지낼까요? 아니면 더 이상의 환경 파괴를 하지 말자고 결정되어 기계를 줄이고 숲과 동물들과 함께 어울려 살아가게 될까요? 외계인과 소통하는 세계가 되어있을 수도 있겠네요! 자신이 생각하는 미래 도시의 이름을 지어주세요. 그리고 미래 도시의 모습을 상상하여 자유롭게 작성해주세요. 그렇게 생각한 이유를 작성해주면 더욱더 좋을 것 같아요.\'}, {\'role\': \'assistant\', \'content\': \'미래의 도시가 어떤 모습일까요? 저는 "에코시티" 라는 이름의 도시를 상상해보고 싶습니다.\\n\\n에코시티는 기존의 도시가 환경을 파괴하는 것과는 반대인 도시입니다. 에코시티는 지구를 보호하고 자연과 함께 살아가는 도시입니다. 에코시티는 기계를 줄이고 자연을 존중하는 도시입니다.\\n\\n에코시티의 주민들은 환경을 보호하는 데에 관심이 많습니다. 도시의 건축물은 자연과 조화를 이룹니다. 건물은 높고 얕은 형태로-designed 되어서 자연의 경관을 훼손하지 않습니다. 도시의 거리에는 많은 숲이 있고, 동물들이 자유롭게 살 수 있는 공간이 많습니다.\\n\\n에코시티의 주민들은 로봇과 자연스럽게 대화할 수 있습니다. 로봇들은 도시의 관리와 유지보수를 도와주고, 주민들의 삶을 편리하게 만듭니다. 하지만 로봇들은 주민들의 삶을 방해하지 않습니다. 주민들은 로봇과 자연스럽게 대화할 수 있게되어, 로봇이 주민들의 삶을 편리하게 만드는 것을 보면서, 로봇과 자연스럽게 대화하는 것이 자연스러워진다는 것을 알 수 있습니다.\\n\\n에코시티의 주민들은 외계인과 소통할 수 있습니다. 에코시티는 외계인과 소통할 수 있는 도시로, 외계인과 교류하는 것이 가능합니다. 에코시티는 세계의 평화를 유지하는 데에 기여할 수 있습니다.\\n\\n에코시티는 기계를 줄이고 자연을 존중하는 도시입니다. 에코시티는 지구를 보호하고 자연과 함께 살아가는 도시입니다. 에코시